### Time-series forecasting | Outline

1. Load the data
2. Explore and plot the data
3. Transform the data
4. Train the model
5. Evaluate the model
6. Saving the model

In [38]:
// ML.NET Nuget packages 
#r "nuget:Microsoft.ML,1.4.0-preview2"  
#r "nuget:Microsoft.ML.FastTree"
#r "nuget:Microsoft.ML.TimeSeries"
    
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

using Microsoft.ML;
using Microsoft.ML.Data;
using XPlot.Plotly;
using Microsoft.ML.Transforms.TimeSeries;

Installing package Microsoft.ML, version 1.4.0-preview2..done!

Successfully added reference to package Microsoft.ML, version 1.4.0-preview2

Installing package Microsoft.ML.FastTree..done!

Successfully added reference to package Microsoft.ML.FastTree, version 1.3.1

Installing package Microsoft.ML.TimeSeries..done!

Successfully added reference to package Microsoft.ML.TimeSeries, version 1.3.1

Installing package XPlot.Plotly, version 2.0.0..done!

Successfully added reference to package XPlot.Plotly, version 2.0.0

In [26]:
public class ProductData
{
    //Next month's #units sold
    [LoadColumn(0)]
    public float next;

    [LoadColumn(1)]
    public float productId;

    [LoadColumn(2)]
    public float year;

    [LoadColumn(3)]
    public float month;

    [LoadColumn(4)]
    public float units;

    //Average #units sold per day in a month
    [LoadColumn(5)]
    public float avg;

    [LoadColumn(6)]
    public float count;

    //Max #units sold per day in a month
    [LoadColumn(7)]
    public float max;

    //Min #units sold per day in a month
    [LoadColumn(8)]
    public float min;

    //Previous month's #units sold
    [LoadColumn(9)]
    public float prev;
}

In [18]:
public class ProductUnitTimeSeriesPrediction
{
    public float[] ForecastedProductUnits { get; set; }

    public float[] ConfidenceLowerBound { get; set; }

    public float[] ConfidenceUpperBound { get; set; }
}

In [19]:
var dataPath = "./Datasets/Time-Series Forecasting/products.stats.csv";

In [29]:
var mlContext = new MLContext(seed: 1);

var dataView = mlContext.Data.LoadFromTextFile<ProductData>(dataPath, hasHeader: true, separatorChar:',');

In [30]:
//Get data for a specific product
var productId = 988;

IDataView productDataView = mlContext.Data.FilterRowsByColumn(dataView, "productId", productId, productId + 1);

#### Data exploration

In [44]:
var rows = mlContext.Data.CreateEnumerable<ProductData>(productDataView, reuseRowObject: false)
                    .Take(5)
                    .ToList();

display(rows);

index,next,productId,year,month,units,avg,count,max,min,prev
0,971,988,2017,1,759,42,18,150,1,427
1,1175,988,2017,2,971,48,20,205,5,759
2,547,988,2017,3,1175,55,21,210,2,971
3,968,988,2017,4,547,28,19,108,6,1175
4,834,988,2017,5,968,40,24,161,5,547


#### Plotting

In [52]:
var years = productDataView.GetColumn<float>("year").ToArray();
var months = productDataView.GetColumn<float>("month").ToArray();

var index = 0;
var dates = new List<DateTime>();
foreach(var year in years)
{
    dates.Add(new DateTime(Convert.ToInt32(year), Convert.ToInt32(months[index]), 1));
    index++;
}

In [87]:
var units = productDataView.GetColumn<float>("units").ToArray();
var max = productDataView.GetColumn<float>("max").ToArray();
var min = productDataView.GetColumn<float>("min").ToArray();
var avg = productDataView.GetColumn<float>("avg").ToArray();

In [112]:
var chart = Chart.Plot(
    new Graph.Scatter()
    {
        x = dates,
        y = units,
        name = "Number of units"         
    },
    new Layout.Layout 
    { 
        title= "Units sold per month",
        yaxis = new Graph.Yaxis { rangemode = "tozero" }
    } 
);

chart.WithLegend(true);
display(chart);

In [103]:
var maxScatter = new Graph.Scatter()
{
    x = dates,
    y = max,
    name = "Max # units sold per day in month"
};

var minScatter = new Graph.Scatter()
{
    x = dates,
    y = min, 
    name = "Min # units sold per day in month"
};
    
var avgScatter = new Graph.Scatter()
{
    x = dates,
    y = avg,
    name = "Average # units sold per day in month"
};    

var scatters = new List<Graph.Scatter> { maxScatter, minScatter, avgScatter };
var layout = new Layout.Layout { title="Time vs Units sold" };

var chart = Chart.Plot(scatters);
chart.WithLayout(new Layout.Layout { title = "Units per day" });
display(chart);

#### Data Transformation

In [8]:
// Create and add the forecast estimator to the pipeline.
IEstimator<ITransformer> forecastEstimator = mlContext.Forecasting.ForecastBySsa(
    outputColumnName: "ForecastedProductUnits",
    inputColumnName: "units",
    windowSize: 12,
    seriesLength: 36,
    trainSize: 36,
    horizon: 2,
    confidenceLevel: 0.95f,
    confidenceLowerBoundColumn: nameof(ProductUnitTimeSeriesPrediction.ConfidenceLowerBound),
    confidenceUpperBoundColumn: nameof(ProductUnitTimeSeriesPrediction.ConfidenceUpperBound));

##### Train the model

In [33]:
ITransformer forecastTransformer = forecastEstimator.Fit(productDataView);

#### Save the model

In [40]:
// Create the forecast engine used for creating predictions.
var forecastEngine = forecastTransformer.CreateTimeSeriesEngine<ProductData, ProductUnitTimeSeriesPrediction>(mlContext);

// Save the forecasting model so that it can be loaded within an end-user app.
forecastEngine.CheckPoint(mlContext, "forecastingEngine.zip");